# Preliminaries

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.utils import resample

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [8]:
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

In [9]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import utils
from sklearn.preprocessing import KernelCenterer, scale
from sklearn.metrics.pairwise import pairwise_kernels
from scipy import linalg
from scipy.sparse.linalg import eigsh as ssl_eigsh

class spca(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_components, kernel="linear", eigen_solver='auto', 
                 max_iterations=None, gamma=0, degree=3, coef0=1, alpha=1.0, 
                 tolerance=0, fit_inverse_transform=False):
        
        self._num_components = num_components
        self._gamma = gamma
        self._tolerance = tolerance
        self._fit_inverse_transform = fit_inverse_transform
        self._max_iterations = max_iterations
        self._degree = degree
        self._kernel = kernel
        self._eigen_solver = eigen_solver
        self._coef0 = coef0
        self._centerer = KernelCenterer()
        self._alpha = alpha
        self._alphas = []
        self._lambdas = []
        
        
    def _get_kernel(self, X, Y=None):
        # Returns a kernel matrix K such that K_{i, j} is the kernel between the ith and jth vectors 
        # of the given matrix X, if Y is None. 
        
        # If Y is not None, then K_{i, j} is the kernel between the ith array from X and the jth array from Y.
        
        # valid kernels are 'linear, rbf, poly, sigmoid, precomputed'
        
        args = {"gamma": self._gamma, "degree": self._degree, "coef0": self._coef0}
        
        return pairwise_kernels(X, Y, metric=self._kernel, n_jobs=-1, filter_params=True, **args)
    
    
    
    def _fit(self, X, Y):
        
        # calculate kernel matrix of the labels Y and centre it and call it K (=H.L.H)
        K = self._centerer.fit_transform(self._get_kernel(Y))
        
        # deciding on the number of components to use
        if self._num_components is not None:
            num_components = min(K.shape[0], self._num_components)
        else:
            num_components = self.K.shape[0]
        
        # Scale X
        # scaled_X = scale(X)
        
        # calculate the eigen values and eigen vectors for X^T.K.X
        Q = (X.T).dot(K).dot(X)
        
        # If n_components is much less than the number of training samples, 
        # arpack may be more efficient than the dense eigensolver.
        if (self._eigen_solver=='auto'):
            if (Q.shape[0]/num_components) > 20:
                eigen_solver = 'arpack'
            else:
                eigen_solver = 'dense'
        else:
            eigen_solver = self._eigen_solver
        
        if eigen_solver == 'dense':
            # Return the eigenvalues (in ascending order) and eigenvectors of a Hermitian or symmetric matrix.
            self._lambdas, self._alphas = linalg.eigh(Q, eigvals=(Q.shape[0] - num_components, Q.shape[0] - 1))
            # argument eigvals = Indexes of the smallest and largest (in ascending order) eigenvalues
        
        elif eigen_solver == 'arpack':
            # deprecated :: self._lambdas, self._alphas = utils.arpack.eigsh(A=Q, num_components, which="LA", tol=self._tolerance)
            self._lambdas, self._alphas = ssl_eigsh(A=Q, k=num_components, which="LA", tol=self._tolerance)
            
        indices = self._lambdas.argsort()[::-1]
        
        self._lambdas = self._lambdas[indices]
        self._lambdas = self._lambdas[self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self._alphas = self._alphas[:, indices]
        #return self._alphas
        self._alphas = self._alphas[:, self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self.X_fit = X

        
    def _transform(self):
        return self.X_fit.dot(self._alphas)
        
        
    def transform(self, X):
        return X.dot(self._alphas)
        
        
    def fit(self, X, Y):
        self._fit(X,Y)
        return
        
        
    def fit_and_transform(self, X, Y):
        self.fit(X, Y)
        return self._transform()
    

In [10]:
df = pd.read_csv('fundamentals.csv')
cols_to_delete = df.columns[df.isnull().sum()/len(df) > .50]
df.drop(cols_to_delete, axis = 1, inplace = True)
df = df.dropna()
df = df.drop(['Unnamed: 0','Ticker Symbol','Period Ending'],axis=1)
normalized_df=(df-df.mean())/df.std()
normalized_df['mean_std'] = normalized_df['Cash Ratio']
normalized_df = normalized_df.drop(['Cash Ratio'],axis=1)
normalized_df

df_trainVal_withTar = normalized_df.iloc[:-442,:]
df_test_withTar = normalized_df.iloc[-442:,:]
df_trainVal = normalized_df.iloc[:-442,:-1]
df_test = normalized_df.iloc[-442:,:-1]
target_df_trainVal = normalized_df.iloc[:-442,-1]
target_df_test = normalized_df.iloc[-442:,-1]

# Regression

# NonLinCFA

In [17]:
#for variable in ['cyclostationary_mean_tg']:#variables_list:
#actual_df_trainVal = df_trainVal_unfolded_std[df_trainVal_unfolded_std.columns[pd.Series(df_trainVal_unfolded_std.columns).str.startswith(variable)]]
for eps in [0.01,0.001,0.0001,0.00001,0.000001]:
    res = []
    nums = []
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = NonLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LinearRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


62
6
1
5
1
Number of aggregated features: 5

0.8136440738302579
58
4
1
10
1
1
Number of aggregated features: 6

0.8157373735623255
62
6
4
1
1
1
Number of aggregated features: 6

0.8136952358922954
62
6
4
1
1
1
Number of aggregated features: 6

0.8059786176772916
63
7
3
1
1
Number of aggregated features: 5

0.8162261979772543
0.813056299787885 +- 0.003234503926486594
5.6 +- 0.42941448508405017
54
6
8
1
4
1
1
Number of aggregated features: 7

0.7935785039702893
50
11
1
5
6
1
1
Number of aggregated features: 7

0.8168243507889493
52
7
8
2
1
3
1
1
Number of aggregated features: 8

0.8075341115270085
48
13
3
5
3
1
1
1
Number of aggregated features: 8

0.7994296314845528
56
8
6
3
1
1
Number of aggregated features: 6

0.813490072905416
0.8061713341352432 +- 0.007570146132932547
7.2 +- 0.6559414608027151
52
8
8
1
3
2
1
Number of aggregated features: 7

0.8180362597089809
50
11
1
5
6
1
1
Number of aggregated features: 7

0.8168243507889492
50
7
6
6
1
3
1
1
Number of aggregated features: 8

0.80

# GenLinCFA

In [29]:
for eps in [0.1,0.15,0.2,0.25,0.3]:
    res = []
    nums = []
    for curr_seed in [0,1,2,3,4]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        mod = LinearRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
        score = mod.score(aggregate_test, df_test_withTar.mean_std)
        print(score)
        res.append(score)
        nums.append(len(output))
    compute_CI(res,5)
    compute_CI(nums,5)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
59
1
2
3
3
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 15

0.8120418172593733
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
61
1
2
3
3
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 13

0.8123928255793366
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
56
1
1
1
1
1
1
2
3
4
5
5
1
1
1
1
1
1
1
1
Number of aggregated features: 16

0.8104174722825184
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
54
1
2
3
4
5
6
6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Number of aggregated features: 17

0.809

# LinCFA

In [31]:
res = []
nums = []
for curr_seed in [0,1,2,3,4]:
    curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
    curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
    curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal), axis=1)
    
    output = LinCFA(curr_df_trainVal_withTar,'mean_std', 0, 0).compute_clusters()
    
    aggregate_trainVal = pd.DataFrame()
    aggregate_test = pd.DataFrame()
    for i in range(len(output)):
        aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
        aggregate_trainVal = aggregate_trainVal.copy()
        aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
        aggregate_test = aggregate_test.copy()
    print(f'Number of aggregated features: {len(output)}\n')
    mod = LinearRegression().fit(aggregate_trainVal, df_trainVal_withTar.mean_std)
    score = mod.score(aggregate_test, df_test_withTar.mean_std)
    print(score)
    res.append(score)
    nums.append(len(output))
compute_CI(res,5)
compute_CI(nums,5)


Number of aggregated features: 11

0.7723522982689838
Number of aggregated features: 10

0.8099957384013146
Number of aggregated features: 12

0.812051816452628
Number of aggregated features: 12

0.8064436887485126
Number of aggregated features: 12

0.8042768415150631
0.8010240766773004 +- 0.012787405976575728
11.4 +- 0.701230917743934


# Baselines

## pca

In [38]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample

r2 = []
n = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    actual_r2 = compare_methods(trainVal_pca, test_pca, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_pca.columns))
    r2.append(actual_r2)
    print(test_pca.shape[1])
    n.append(test_pca.shape[1])
compute_CI(r2,5)
compute_CI(n,5)
    

Full aggregate regression train score: 0.922011320032613, test score: 0.7593726250943146
Aggregate regression train score with FS: 0.922011320032613, test score: 0.7593726250943146
27
Full aggregate regression train score: 0.9058861660878511, test score: 0.7525871350536726
Aggregate regression train score with FS: 0.9058861660878511, test score: 0.7525871350536726
26
Full aggregate regression train score: 0.907905279422283, test score: 0.7546866705898678
Aggregate regression train score with FS: 0.907905279422283, test score: 0.7546866705898678
27
Full aggregate regression train score: 0.9150762209326956, test score: 0.75309811827884
Aggregate regression train score with FS: 0.9150762209326956, test score: 0.75309811827884
26
Full aggregate regression train score: 0.871621141464457, test score: 0.7599120858347211
Aggregate regression train score with FS: 0.871621141464457, test score: 0.7599120858347211
27
0.7559313269702832 +- 0.002728523722143103
26.6 +- 0.42941448508405017


## kernel pca

In [41]:
from sklearn.decomposition import KernelPCA
for curr_kernel in ['linear', 'poly', 'sigmoid']:
    best_scores = []
    best_nums = []
    for curr_seed in [0,1,2,3,4]: 
        best_score = 0
        best_num = 0
        print(f'#########################{curr_kernel}, {curr_seed}#############################')
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################linear, 0#############################
(442, 1)
-0.0018544102584430178
(442, 2)
-0.003280726489439667
(442, 3)
0.09567341254614958
(442, 4)
0.09403905939583246
(442, 5)
0.36430035597335564
(442, 6)
0.3620050578937106
(442, 7)
0.32649790067559636
(442, 8)
0.46733819536579924
(442, 9)
0.4833238430001322
(442, 10)
0.502637863616437
(442, 11)
0.530014093557158
(442, 12)
0.7106851966991402
(442, 13)
0.6963421061143209
(442, 14)
0.6973603195858842
(442, 15)
0.7012568986441079
(442, 16)
0.7048578221301454
(442, 17)
0.7146307600642655
(442, 18)
0.7166811477109108
(442, 19)
0.7169235074194249
(442, 20)
0.7495553815757968
(442, 21)
0.7494723855356651
(442, 22)
0.7537138463111344
(442, 23)
0.7648184532458546
(442, 24)
0.7648008042456759
(442, 25)
0.7636537759973359
(442, 26)
0.760333447297009
(442, 27)
0.7593726250943142
(442, 28)
0.7635576567400576
(442, 29)
0.7653090569666516
(442, 30)
0.7616369883929337
(442, 31)
0.736858498004758
(442, 32)
0.7382039366858885
(442, 33)

(442, 15)
-0.004036303966620025
(442, 16)
-0.00400907005153428
(442, 17)
-0.020178199852025402
(442, 18)
-0.017469478795663074
(442, 19)
-0.0034352462211462154
(442, 20)
-0.17163691410732151
(442, 21)
-0.12020191487664067
(442, 22)
-0.44708191055586566
(442, 23)
-0.36543314409796324
(442, 24)
0.505889663551473
(442, 25)
0.4987617263343981
(442, 26)
0.5917763703773973
(442, 27)
0.6203517886149581
(442, 28)
0.6244514570849583
(442, 29)
0.6425644938259301
(442, 30)
0.7217306717187605
(442, 31)
0.7202095107498279
(442, 32)
0.7119475665252637
(442, 33)
0.711597269209181
(442, 34)
0.7019280462245097
(442, 35)
0.7012065382100884
(442, 36)
0.558008124197727
(442, 37)
0.5846600364686916
(442, 38)
0.6656481360377622
(442, 39)
0.6413478390934688
(442, 40)
0.707113332535654
(442, 41)
0.6449054704942341
(442, 42)
0.6437006287188634
(442, 43)
0.673576142816842
(442, 44)
0.5967023210256899
(442, 45)
0.5346782496464071
(442, 46)
0.543406890276595
(442, 47)
0.41379097631448714
(442, 48)
0.3947604238614

(442, 20)
0.03698481425835498
(442, 21)
0.03598975698246787
(442, 22)
0.11406353714135331
(442, 23)
0.09044374903278507
(442, 24)
0.12034396480256537
(442, 25)
0.10446599660344791
(442, 26)
0.06898925019687419
(442, 27)
0.02757056442105743
(442, 28)
-0.004994856303079276
(442, 29)
0.02133051315016865
(442, 30)
-0.007394918311959042
(442, 31)
0.024052962664159794
(442, 32)
0.04204759670158287
(442, 33)
0.043277362081020065
(442, 34)
0.01909520286374644
(442, 35)
0.0953163485215095
(442, 36)
0.1391027986393344
(442, 37)
0.046131822094679586
(442, 38)
0.0674049486959879
(442, 39)
0.08673758803220655
(442, 40)
0.004845138164390694
(442, 41)
-0.026462813573504995
(442, 42)
-0.3044604910766424
(442, 43)
-0.7879783158013232
(442, 44)
-0.7996868120059353
(442, 45)
-0.8898548946433198
(442, 46)
-0.8428276671617858
(442, 47)
-1.0294719155140881
(442, 48)
-1.050754089048909
(442, 49)
-1.3981128413081678
(442, 50)
-1.3759221358033291
0.415131366533506
(442, 2)
#########################sigmoid, 1##

In [ ]:
0.7763803385484433 +- 0.01176161585485915
36.0 +- 10.806694221638733

0.7105455478106365 +- 0.03536120629257586
41.0 +- 5.1110546856788766

0.4349818403914211 +- 0.02364046317279162
3.4 +- 1.188994869627283

## isomap

In [48]:
from sklearn.manifold import Isomap
best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(442, 1)
-0.005760793109692575


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(442, 2)
0.0020468006205934808


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(442, 48)
-0.17775801319203532


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(442, 49)
-0.1639089977803465
(442, 50)
-0.22099693539536647
0.26785424543213054
34
0.26101532597218985 +- 0.04581068695546693
27.2 +- 6.821250559831386


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


In [ ]:
0.26101532597218985 +- 0.04581068695546693
27.2 +- 6.8212505598313860

## LLE

In [47]:
from sklearn.manifold import LocallyLinearEmbedding as LLE

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(442, 1)
0.5445025801660901
(442, 2)
0.5462416922307325
(442, 3)
0.5464474327223566
(442, 4)
0.5432544351958116


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(442, 5)
0.6069821525130226
(442, 6)
0.6712136599154954
(442, 7)
0.6712486553707209
(442, 8)
0.6691208816047118


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(442, 46)
0.6930429760064416
(442, 47)
0.6967905479086233


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(442, 48)
0.6968845729396104
(442, 49)
0.6955017653511779
(442, 50)
0.6917718195718925
0.6968845729396104
48
0.7280457662557442 +- 0.02668950852105602
44.0 +- 7.270516900468631


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


In [ ]:
0.7280457662557442 +- 0.02668950852105602
44.0 +- 7.270516900468631

## Sup PCA

In [75]:
for kernel in ['sigmoid']:#['poly']:#['linear']:
    best_scores = []
    best_nums = []
    for curr_seed in [0,3,8,12,5]: 
        print(f'#########################{kernel}, {curr_seed}#############################')
        best_score = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel=kernel, degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################sigmoid, 0#############################
(442, 1)
0.694209268415197
(442, 2)
0.5954490761978187
(442, 3)
0.5638963926824548
(442, 4)
0.5130217415656153
(442, 5)
0.5130207860929714
(442, 6)
0.5497002112132319
(442, 7)
0.606245994116924
(442, 8)
0.5956382683118397
(442, 9)
0.51155709223451
(442, 10)
0.501054509768553
(442, 11)
0.5222448764327459
(442, 12)
0.5211831490424863
(442, 13)
0.5644962755944005
(442, 14)
0.543720460633436
(442, 15)
0.5667410624457011
(442, 16)
0.48867076418902855
(442, 17)
0.5247202609558146
(442, 18)
0.5333718944831355
(442, 19)
0.5111876649065379
(442, 20)
0.3520527060274632
(442, 21)
0.3708635697127567
(442, 22)
0.33269473424686835
(442, 23)
0.13720356057849104
(442, 24)
0.11192837806518308
(442, 25)
0.09387775592724734
(442, 26)
0.05498273807552112
(442, 27)
0.21185183380016925
(442, 28)
0.18317335908685917
(442, 29)
0.20944716924831142
(442, 30)
0.09429999865202587
(442, 31)
0.32741193642730304
(442, 32)
0.31963052114482926
(442, 33)


In [ ]:
0.773055927916064 +- 0.012552027829434004
31.0 +- 13.77030776707623

0.7626988470258912 +- 0.01156512478454842
26.0 +- 12.246471491821634

0.6409896167884936 +- 0.03842632477566533
16.0 +- 13.247048274993189

## UMAP

In [12]:
import umap

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = umap.UMAP(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(442, 1)
0.08810835121279548
(442, 2)
0.330757000367491
(442, 3)
0.26659863625343816
(442, 4)
0.3180765813605223
(442, 5)
0.2816786939926871
(442, 6)
0.47255594405946355
(442, 7)
0.3591710544765149
(442, 8)
0.35350826937058133
(442, 9)
0.450469023343672
(442, 10)
0.321923343605095
(442, 11)
0.40699801515079437
(442, 12)
0.39455519397994054
(442, 13)
0.47543490678157996
(442, 14)
0.4484876742950098
(442, 15)
0.43696096077107105
(442, 16)
0.2730611326616482
(442, 17)
0.4819371337921463
(442, 18)
0.5251711860067776
(442, 19)
0.468022789732929
(442, 20)
0.4736760996373073
(442, 21)
0.5260779490427746
(442, 22)
0.49488076613877763
(442, 23)
0.5542597447269181
(442, 24)
0.5263901967791711
(442, 25)
0.5100943023749833
(442, 26)
0.5424001784675228
(442, 27)
0.5265398602615985
(442, 28)
0.5238314714369225
(442, 29)
0.44531783651313084
(442, 30)
0.5354238080030431
(442, 31)
0.49611702026765314
(442, 32)
0.508730434297642
(442, 33)
0.5357498187650893
(442, 34)
0.45819036684543646
(442, 35)
0.5300

In [ ]:
0.626848326328787 +- 0.02482459870273499
44.8 +- 4.236502708602934

## t-sne

In [18]:
from sklearn.manifold import TSNE

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = TSNE(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        
        mod = LinearRegression().fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        
        trainValTest = pd.concat((curr_df_trainVal,df_test),axis=0)
        test_reduced = pd.DataFrame(dimRedMethod.fit_transform(trainValTest.values)).iloc[-df_test.shape[0]:,:]
        print(test_reduced.shape)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape[1]
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num)
compute_CI(best_scores,5)
compute_CI(best_nums,5)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


(442, 1)
-0.9849195121116525


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


(442, 2)
0.1850353876146149


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


(442, 3)
-0.5316912175724298


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


ValueError: 'n_components' should be inferior to 4 for the barnes_hut algorithm as it relies on quad-tree or oct-tree.